In [ ]:
!pip3 install transformers

In [ ]:
import json

# Load the dataset
def load_squad_format(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
       data = json.load(f)

    contexts = []
    questions = []
    answers = []

    for item in data["data"]:
        for paragraph in item["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                for answer in qa["answers"]:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer["text"])

    return contexts, questions, answers

# Load train and validation datasets
train_contexts, train_questions, train_answers = load_squad_format('/content/train.json')
val_contexts, val_questions, val_answers = load_squad_format('/content/val.json')
test_contexts, test_questions, test_answers = load_squad_format('/content/test.json')


In [ ]:
import pandas as pd
df_train=pd.DataFrame()
df_train["context"]=train_contexts
df_train["questions"]=train_questions
df_train["answers"]=train_answers


df_test=pd.DataFrame()
df_test["context"]=test_contexts
df_test["questions"]=test_questions
df_test["answers"]=test_answers

df_val=pd.DataFrame()
df_val["context"]=val_contexts
df_val["questions"]=val_questions
df_val["answers"]=val_answers

df_val.head(5)

,context,questions,answers
0,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,ما هي غابات ميومبو وأين توجد؟\n,هي أراضي عُشبية استوائية وشبه استوائية بيولوجي...
1,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,ما هي المناطق البيئية الأربعة التي تميز غابات ...,يُمكن تصنيف غابات ميومبو على أنها جافةً أو رط...
2,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,كيف يتم تصنيف غابات ميومبو إلى أغاب ورطبة؟\n,هي تلك التي تستقبل أكثر من 1000 مم سنويًا لهطو...
3,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,كيف تساعد غابات ميومبو في توفير سُبل العيش للس...,الذين يعتمدون على الموارد المتاحة من الغابات
4,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,ما هي المنتجات غير الخشبية التي يمكن العثور عل...,مثل الفاكهة والعسل وعلف الماشية وحطب الوقود


In [ ]:
from transformers import AutoTokenizer

# Load GPT-2 tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")



       # Add a new [PAD] token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})


def tokenize_data(contexts, questions, answers, tokenizer, max_length=512):
    input_texts = [f"السؤال: {q} السياق: {c}" for q, c in zip(questions, contexts)]
    outputs = answers

    inputs = tokenizer(
        input_texts,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    outputs = tokenizer(
        outputs,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Set labels to outputs for GPT-2 fine-tuning
    inputs["labels"] = outputs["input_ids"]
    return inputs

def tokenize_test_data(contexts, questions, tokenizer, max_length=512):
    input_texts = [f"السؤال: {q} السياق: {c}" for q, c in zip(questions, contexts)]

    # Now tokenize the data
    inputs = tokenizer(
        input_texts,
        max_length=128,
        truncation=True,
        padding="max_length",  # Padding is now enabled
        return_tensors="pt"
    )

    outputs = tokenizer(
        outputs,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    return inputs

# Tokenize train and validation data
train_data = tokenize_data(train_contexts, train_questions, train_answers, tokenizer)
val_data = tokenize_data(val_contexts, val_questions, val_answers, tokenizer)
#test_data = tokenize_test_data(test_contexts, test_questions, test_answers, tokenizer)





/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(f"Contexts: {len(train_contexts)}")
print(f"Questions: {len(train_questions)}")
print(f"Answers: {len(train_answers)}")

Contexts: 64782
Questions: 64782
Answers: 64782


In [ ]:
print(f"Input IDs shape: {train_data['input_ids'].shape}")
print(f"Labels shape: {train_data['labels'].shape}")

Input IDs shape: torch.Size([64782, 128])
Labels shape: torch.Size([64782, 128])


In [ ]:
import torch
from torch.utils.data import Dataset
class QADataset(Dataset):
    def __init__(self, inputs):
        self.input_ids = inputs["input_ids"]
        self.attention_mask = inputs["attention_mask"]
        self.labels = inputs["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

# Create datasets
train_dataset = QADataset(train_data)
val_dataset = QADataset(val_data)



In [ ]:
from transformers import GPT2LMHeadModel

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("aubmindlab/aragpt2-base")

model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(64001, 768)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,  # Ensure the model aligns with the tokenizer
    padding="max_length",  # Force padding to the max sequence length
)

In [ ]:
from transformers import Trainer, TrainingArguments, TrainerCallback
import torch

class TrainingMonitorCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"\n=== Starting Epoch: {state.epoch + 1}/{int(args.num_train_epochs)} ===\n")

    def on_step_begin(self, args, state, control, **kwargs):
        print(f"--- Step: {state.global_step}/{state.max_steps} ---")

    def on_train_batch_begin(self, args, state, control, train_dataloader, **kwargs):
        print(f"Processing Batch {state.global_step}...")

def setup_trainer(model, train_dataset, val_dataset, tokenizer):
    """
    Setup the trainer with proper configuration
    """
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        # Make batch sizes consistent
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        logging_dir="./logs",
        logging_steps=10,
        save_steps=500,
        evaluation_strategy="steps",  # Fixed typo in eval_strategy
        eval_steps=100,
        # Add gradient accumulation to handle memory
        gradient_accumulation_steps=4,
        # Add warmup steps
        warmup_steps=500,
        # Add weight decay
        weight_decay=0.01,
        # Add learning rate
        learning_rate=5e-5,
        # Prevent memory issues
        fp16=True,  # Use mixed precision training
        dataloader_pin_memory=False,  # Prevent memory issues
        remove_unused_columns=False  # Keep all columns
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        callbacks=[TrainingMonitorCallback()],
        # Add data collator if needed
        data_collator=data_collator
    )

    return trainer

# Verify dataset shapes before training
def verify_datasets(train_dataset, val_dataset):
    print("Training Dataset:")
    sample = train_dataset[0]
    for key, value in sample.items():
        print(f"{key} shape: {value.shape}")

    print("\nValidation Dataset:")
    sample = val_dataset[0]
    for key, value in sample.items():
        print(f"{key} shape: {value.shape}")

# Usage
def train_model(model, train_dataset, val_dataset, tokenizer):
    # Verify datasets
    verify_datasets(train_dataset, val_dataset)

    # Setup trainer
    trainer = setup_trainer(model, train_dataset, val_dataset, tokenizer)

    # Train model
    try:
        trainer.train()
    except Exception as e:
        print(f"Training error: {str(e)}")
        # Print shapes of the batch causing the error
        print("\nDebug information:")
        print(f"Train dataset size: {len(train_dataset)}")
        print(f"Val dataset size: {len(val_dataset)}")
        raise e

train_model(model, train_dataset, val_dataset, tokenizer)

Training Dataset:
input_ids shape: torch.Size([128])
attention_mask shape: torch.Size([128])
labels shape: torch.Size([128])

Validation Dataset:
input_ids shape: torch.Size([128])
attention_mask shape: torch.Size([128])
labels shape: torch.Size([128])


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-0d58259c8e98>:44: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.



=== Starting Epoch: 1/1 ===

--- Step: 0/2024 ---


/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
100,45.445600,11.095170
200,41.791800,10.174599
300,6.290300,1.255100
400,4.319100,1.016378
500,3.883300,0.985823
600,4.117400,0.965380
700,3.472800,0.968091
800,3.866700,0.948691
900,3.619900,0.945851
1000,3.691700,0.941807


--- Step: 1/2024 ---
--- Step: 2/2024 ---
--- Step: 3/2024 ---
--- Step: 4/2024 ---
--- Step: 5/2024 ---
--- Step: 6/2024 ---
--- Step: 7/2024 ---
--- Step: 8/2024 ---
--- Step: 9/2024 ---
--- Step: 10/2024 ---
--- Step: 11/2024 ---
--- Step: 12/2024 ---
--- Step: 13/2024 ---
--- Step: 14/2024 ---
--- Step: 15/2024 ---
--- Step: 16/2024 ---
--- Step: 17/2024 ---
--- Step: 18/2024 ---
--- Step: 19/2024 ---
--- Step: 20/2024 ---
--- Step: 21/2024 ---
--- Step: 22/2024 ---
--- Step: 23/2024 ---
--- Step: 24/2024 ---
--- Step: 25/2024 ---
--- Step: 26/2024 ---
--- Step: 27/2024 ---
--- Step: 28/2024 ---
--- Step: 29/2024 ---
--- Step: 30/2024 ---
--- Step: 31/2024 ---
--- Step: 32/2024 ---
--- Step: 33/2024 ---
--- Step: 34/2024 ---
--- Step: 35/2024 ---
--- Step: 36/2024 ---
--- Step: 37/2024 ---
--- Step: 38/2024 ---
--- Step: 39/2024 ---
--- Step: 40/2024 ---
--- Step: 41/2024 ---
--- Step: 42/2024 ---
--- Step: 43/2024 ---
--- Step: 44/2024 ---
--- Step: 45/2024 ---
--- Step: 46/2024 -

So far , we tried 2 and 3 epochs but we kept facing the OutOfMemoryError. 

In [ ]:
from transformers import AutoModelForCausalLM, pipeline


# Create the pipeline
qa_pipeline = pipeline(
    task="text-generation",
    model=saved_model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

# Example usage
context = "هذا نص يحتوي على معلومات حول البرمجيات مفتوحة المصدر"
question ="ما هي غابات ميومبو وأين توجد؟"
input_text = f"{question} {context}"

response = qa_pipeline(
    input_text,
    max_length=150,
    num_return_sequences=1,
    pad_token_id=tokenizer.pad_token_id
)
print(response[0]['generated_text'])

Device set to use cuda:0


ما هي البرمجيات مفتوحة المصدر؟ هذا نص يحتوي على معلومات حول البرمجيات مفتوحة المصدر.ٌ��� ( �»�)�ا « من في. الإنترنت الحاسوب� استخدام غير"ةر
 هذهح الفيديو� إلى و حتىولس 2/ مايكروسوفت ما مع كان ر


In [ ]:

from transformers import AutoModelForCausalLM
saved_model = AutoModelForCausalLM.from_pretrained("./results/checkpoint-2024")


In [ ]:
from transformers import Trainer, TrainingArguments
import torch
import numpy as np

def evaluate_model(model, val_dataset, tokenizer):
    """
    Evaluation function that computes loss, perplexity, and accuracy

    Args:
        model: The trained model to evaluate
        val_dataset: Validation dataset
        tokenizer: The tokenizer used

    Returns:
        dict: Dictionary containing evaluation metrics
    """

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred

        # Shift predictions and labels for causal LM
        shifted_predictions = predictions[:, :-1, :]
        shifted_labels = labels[:, 1:]

        # Get the most likely token predictions
        pred_tokens = shifted_predictions.argmax(axis=-1)

        # Calculate accuracy
        # Create mask for non-padding tokens (assuming pad_token_id is 0 or a specific value)
        mask = (shifted_labels != tokenizer.pad_token_id).flatten()

        # Calculate token-level accuracy
        correct_tokens = (pred_tokens == shifted_labels).flatten()[mask]
        accuracy = correct_tokens.sum().item() / mask.sum().item()

        # Calculate loss and perplexity
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(shifted_predictions.view(-1, shifted_predictions.size(-1)),
                       shifted_labels.view(-1))
        perplexity = torch.exp(loss)

        # Calculate accuracy for non-padding tokens only
        non_pad_mask = shifted_labels != tokenizer.pad_token_id
        total_tokens = non_pad_mask.sum().item()
        correct_tokens = ((pred_tokens == shifted_labels) & non_pad_mask).sum().item()
        accuracy = correct_tokens / total_tokens if total_tokens > 0 else 0

        return {
            "perplexity": perplexity.item(),
            "loss": loss.item(),
            "accuracy": accuracy,
            "total_tokens": total_tokens,
            "correct_tokens": correct_tokens
        }

    # Setup evaluation-specific training arguments
    eval_args = TrainingArguments(
        output_dir="./eval_results",
        per_device_eval_batch_size=8,
        eval_accumulation_steps=4,
        fp16=True,
        dataloader_pin_memory=False,
        remove_unused_columns=False,
        report_to="none"  # Disable wandb or other logging
    )

    # Initialize trainer for evaluation only
    eval_trainer = Trainer(
        model=model,
        args=eval_args,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    try:
        print("Starting evaluation...")
        eval_results = eval_trainer.evaluate()

        print("\nEvaluation Results:")
        print(f"Loss: {eval_results['eval_loss']:.4f}")
        print(f"Perplexity: {eval_results['eval_perplexity']:.4f}")
        print(f"Accuracy: {eval_results['eval_accuracy']*100:.2f}%")
        print(f"Total Tokens: {eval_results['eval_total_tokens']}")
        print(f"Correct Tokens: {eval_results['eval_correct_tokens']}")

        return eval_results

    except Exception as e:
        print(f"Evaluation error: {str(e)}")
        print(f"Validation dataset size: {len(val_dataset)}")
        raise e


In [ ]:

eval_results = evaluate_model(saved_model, val_dataset, tokenizer)

<ipython-input-2-02441251f1d9>:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(


Starting evaluation...


MultiHeadDifferentialAttention

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MultiHeadDifferentialAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1, lambda_init=0.8):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_head = d_model // (2 * num_heads)  # divide by 2 because we have two sets of Q,K
        
        # Linear projections for Q1, Q2, K1, K2, V
        self.W_Q = nn.Linear(d_model, 2 * d_model)  # For both Q1 and Q2
        self.W_K = nn.Linear(d_model, 2 * d_model)  # For both K1 and K2
        self.W_V = nn.Linear(d_model, 2 * d_model)
        
        # Learnable vectors for lambda calculation
        self.lambda_q1 = nn.Parameter(torch.randn(self.d_head))
        self.lambda_k1 = nn.Parameter(torch.randn(self.d_head))
        self.lambda_q2 = nn.Parameter(torch.randn(self.d_head))
        self.lambda_k2 = nn.Parameter(torch.randn(self.d_head))
        self.lambda_init = lambda_init
        
        self.dropout = nn.Dropout(dropout)
        self.output_linear = nn.Linear(d_model, d_model)

    def compute_lambda(self):
        # Implement lambda calculation as per equation (2)
        lambda_term1 = torch.exp(torch.dot(self.lambda_q1, self.lambda_k1))
        lambda_term2 = torch.exp(torch.dot(self.lambda_q2, self.lambda_k2))
        return lambda_term1 - lambda_term2 + self.lambda_init

    def split_heads(self, x, is_values=False):
        batch_size = x.size(0)
        seq_length = x.size(1)
        
        # Split for values is different as V dimension is 2*d
        feature_size = x.size(-1) // (2 if not is_values else 1)
        
        # Reshape and transpose for multi-head attention
        if not is_values:
            x = x.view(batch_size, seq_length, self.num_heads, 2, self.d_head)
            # Separate Q1,Q2 or K1,K2
            x1, x2 = x[:, :, :, 0, :], x[:, :, :, 1, :]
            # Reshape for attention computation
            x1 = x1.transpose(1, 2)
            x2 = x2.transpose(1, 2)
            return x1, x2
        else:
            x = x.view(batch_size, seq_length, self.num_heads, self.d_head * 2)
            return x.transpose(1, 2)

    def forward(self, x, mask=None):
        batch_size = x.size(0)
        
        # Linear projections
        q = self.W_Q(x)
        k = self.W_K(x)
        v = self.W_V(x)
        
        # Split heads
        q1, q2 = self.split_heads(q)
        k1, k2 = self.split_heads(k)
        v = self.split_heads(v, is_values=True)
        
        # Scale factor
        scale = math.sqrt(self.d_head)
        
        # Compute lambda
        lambda_val = self.compute_lambda()
        
        # Compute attention scores
        scores1 = torch.matmul(q1, k1.transpose(-2, -1)) / scale
        scores2 = torch.matmul(q2, k2.transpose(-2, -1)) / scale
        
        if mask is not None:
            scores1 = scores1.masked_fill(mask == 0, -1e9)
            scores2 = scores2.masked_fill(mask == 0, -1e9)
        
        # Apply differential attention formula
        attn1 = F.softmax(scores1, dim=-1)
        attn2 = F.softmax(scores2, dim=-1)
        
        attn1 = self.dropout(attn1)
        attn2 = self.dropout(attn2)
        
        # Compute differential attention output
        out = torch.matmul(attn1, v) - lambda_val * torch.matmul(attn2, v)
        
        # Reshape and apply output projection
        out = out.transpose(1, 2).contiguous()
        out = out.view(batch_size, -1, self.d_model)
        out = self.output_linear(out)
        
        return out

import random
from transformers import AutoModel

def replace_attention_layers(model, replacement_ratio=0.35):
    """
    Replace a portion of the model's attention layers with differential attention
    Args:
        model: The original model
        replacement_ratio: Percentage of layers to replace (between 0.25 and 0.5)
    """
    # Ensure replacement ratio is within bounds
    replacement_ratio = max(0.25, min(0.5, replacement_ratio))
    
    # Get all attention layers
    attention_layers = []
    for name, module in model.named_modules():
        if "attention" in name.lower() and "output" not in name.lower():
            attention_layers.append(name)
    
    # Calculate number of layers to replace
    num_to_replace = int(len(attention_layers) * replacement_ratio)
    layers_to_replace = random.sample(attention_layers, num_to_replace)
    
    # Replace selected layers
    for layer_name in layers_to_replace:
        # Get the original layer to match dimensions
        original_layer = dict(model.named_modules())[layer_name]
        config = original_layer.config if hasattr(original_layer, 'config') else None
        
        # Create new differential attention layer
        new_layer = MultiHeadDifferentialAttention(
            d_model=config.hidden_size if config else 768,  # default to 768 if config not available
            num_heads=config.num_attention_heads if config else 12,
            dropout=config.attention_probs_dropout_prob if config else 0.1
        )
        
        # Recursively set the new layer
        parent = model
        for part in layer_name.split('.')[:-1]:
            parent = getattr(parent, part)
        setattr(parent, layer_name.split('.')[-1], new_layer)
    
    return model

def modify_and_train_model(original_model, train_dataset, val_dataset, tokenizer):
    """
    Modify the model with differential attention and train it
    """
    # Replace attention layers
    modified_model = replace_attention_layers(original_model)
    
    # Train the modified model
    trainer = setup_trainer(modified_model, train_dataset, val_dataset, tokenizer)
    results = trainer.train()
    
    return modified_model, trainer, results


In [ ]:
# Load your original model
original_model = AutoModel.from_pretrained("aubmindlab/aragpt2-base")

# Modify and train the model
modified_model, trainer, results = modify_and_train_model(
    original_model,
    train_dataset,
    val_dataset,
    tokenizer
)